In [1]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
from sqlalchemy import create_engine
import numpy as np
import pandas as pd
import requests
import pymongo
from datetime import datetime
import ast

In [2]:
file = "data/Denver/crime.csv"
data1 = pd.read_csv(file)

In [3]:
# Column Names from csv
# ----------------------------
# INCIDENT_ID	OFFENSE_ID	OFFENSE_CODE	OFFENSE_CODE_EXTENSION	OFFENSE_TYPE_ID	OFFENSE_CATEGORY_ID	FIRST_OCCURRENCE_DATE	
# LAST_OCCURRENCE_DATE	REPORTED_DATE	INCIDENT_ADDRESS	GEO_X	GEO_Y	GEO_LON	GEO_LAT	DISTRICT_ID	PRECINCT_ID	NEIGHBORHOOD_ID	IS_CRIME	IS_TRAFFIC
data = data1[["INCIDENT_ID", "OFFENSE_CODE", "FIRST_OCCURRENCE_DATE", "OFFENSE_TYPE_ID", "OFFENSE_CATEGORY_ID", "GEO_LAT", "GEO_LON"]].copy()
data.rename(columns={'GEO_LAT': 'latitude', 'GEO_LON': 'longitude'}, inplace=True)
data["city"] = "denver"
data

,INCIDENT_ID,OFFENSE_CODE,FIRST_OCCURRENCE_DATE,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,latitude,longitude,city
0,2016376978,5213,6/15/2016 11:31:00 PM,weapon-unlawful-discharge-of,all-other-crimes,39.773188,-104.809881,denver
1,20186000994,2399,10/11/2017 12:30:00 PM,theft-other,larceny,39.785649,-104.781434,denver
2,20166003953,2305,3/4/2016 8:00:00 PM,theft-items-from-vehicle,theft-from-motor-vehicle,39.663490,-104.957381,denver
3,201872333,2399,1/30/2018 7:20:00 PM,theft-other,larceny,39.702698,-104.941440,denver
4,2017411405,2303,6/22/2017 8:53:00 PM,theft-shoplift,larceny,39.717107,-104.955370,denver
5,201872837,5499,1/31/2018 12:44:00 AM,traf-other,all-other-crimes,39.743149,-104.961928,denver
6,20186001015,2304,6/1/2017 12:15:00 PM,theft-parts-from-vehicle,theft-from-motor-vehicle,39.732790,-105.025543,denver
7,201870628,5707,1/30/2018 7:40:00 AM,criminal-trespassing,all-other-crimes,39.723424,-104.983794,denver
8,201870833,5401,1/30/2018 9:10:00 AM,traffic-accident-hit-and-run,traffic-accident,39.736863,-104.991650,denver
9,20186001048,2305,1/31/2018 12:55:00 AM,theft-items-from-vehicle,theft-from-motor-vehicle,39.757627,-105.015451,denver


In [4]:
dates = []
years = []

for date in data["FIRST_OCCURRENCE_DATE"]:
    date = datetime.strptime(date, "%m/%d/%Y %I:%M:%S %p")
    dates.append(date)
    years.append(date.year)
    
data["date"] = dates
data["years"] = years

data = data.loc[(data["years"] >= 2014) & (data["years"] <= 2018)]
data


,INCIDENT_ID,OFFENSE_CODE,FIRST_OCCURRENCE_DATE,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,latitude,longitude,city,date,years
0,2016376978,5213,6/15/2016 11:31:00 PM,weapon-unlawful-discharge-of,all-other-crimes,39.773188,-104.809881,denver,2016-06-15 23:31:00,2016
1,20186000994,2399,10/11/2017 12:30:00 PM,theft-other,larceny,39.785649,-104.781434,denver,2017-10-11 12:30:00,2017
2,20166003953,2305,3/4/2016 8:00:00 PM,theft-items-from-vehicle,theft-from-motor-vehicle,39.663490,-104.957381,denver,2016-03-04 20:00:00,2016
3,201872333,2399,1/30/2018 7:20:00 PM,theft-other,larceny,39.702698,-104.941440,denver,2018-01-30 19:20:00,2018
4,2017411405,2303,6/22/2017 8:53:00 PM,theft-shoplift,larceny,39.717107,-104.955370,denver,2017-06-22 20:53:00,2017
5,201872837,5499,1/31/2018 12:44:00 AM,traf-other,all-other-crimes,39.743149,-104.961928,denver,2018-01-31 00:44:00,2018
6,20186001015,2304,6/1/2017 12:15:00 PM,theft-parts-from-vehicle,theft-from-motor-vehicle,39.732790,-105.025543,denver,2017-06-01 12:15:00,2017
7,201870628,5707,1/30/2018 7:40:00 AM,criminal-trespassing,all-other-crimes,39.723424,-104.983794,denver,2018-01-30 07:40:00,2018
8,201870833,5401,1/30/2018 9:10:00 AM,traffic-accident-hit-and-run,traffic-accident,39.736863,-104.991650,denver,2018-01-30 09:10:00,2018
9,20186001048,2305,1/31/2018 12:55:00 AM,theft-items-from-vehicle,theft-from-motor-vehicle,39.757627,-105.015451,denver,2018-01-31 00:55:00,2018




----

# Data Munging

In [6]:
# NA NAN Checking
num_NA = data.isna().sum()
print(num_NA)
print("\nPercentage of NA in whole data: \n" + str(num_NA/data["INCIDENT_ID"].count()))
      

INCIDENT_ID                 0
OFFENSE_CODE                0
FIRST_OCCURRENCE_DATE       0
OFFENSE_TYPE_ID             0
OFFENSE_CATEGORY_ID         0
latitude                 3723
longitude                3723
city                        0
date                        0
years                       0
dtype: int64

Percentage of NA in whole data: 
INCIDENT_ID              0.000000
OFFENSE_CODE             0.000000
FIRST_OCCURRENCE_DATE    0.000000
OFFENSE_TYPE_ID          0.000000
OFFENSE_CATEGORY_ID      0.000000
latitude                 0.008266
longitude                0.008266
city                     0.000000
date                     0.000000
years                    0.000000
dtype: float64


In [7]:
# Identify rows with Nan in lat/long
data_NA = data.loc[data["latitude"].isna()]
data_NA

,INCIDENT_ID,OFFENSE_CODE,FIRST_OCCURRENCE_DATE,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,latitude,longitude,city,date,years
177,201874540,1109,1/27/2018 10:00:00 PM,sex-aslt-non-rape,sexual-assault,NaN,NaN,denver,2018-01-27 22:00:00,2018
213,201869284,1109,1/29/2018 1:00:00 PM,sex-aslt-non-rape,sexual-assault,NaN,NaN,denver,2018-01-29 13:00:00,2018
258,2017787782,1102,11/8/2017 10:00:00 PM,sex-aslt-rape,sexual-assault,NaN,NaN,denver,2017-11-08 22:00:00,2017
297,201873793,1102,1/30/2018 11:00:00 AM,sex-aslt-rape,sexual-assault,NaN,NaN,denver,2018-01-30 11:00:00,2018
335,20185000555,1102,12/1/2017 7:00:00 AM,sex-aslt-rape-pot,sexual-assault,NaN,NaN,denver,2017-12-01 07:00:00,2017
414,201872476,1102,1/20/2018 9:01:00 PM,sex-aslt-rape,sexual-assault,NaN,NaN,denver,2018-01-20 21:01:00,2018
436,201844598,1102,1/17/2018 11:45:00 PM,sex-aslt-rape,sexual-assault,NaN,NaN,denver,2018-01-17 23:45:00,2018
470,201872530,1102,1/30/2018 8:00:00 PM,sex-aslt-rape,sexual-assault,NaN,NaN,denver,2018-01-30 20:00:00,2018
581,20185000508,1102,1/17/2018 12:00:00 PM,sex-aslt-rape-pot,sexual-assault,NaN,NaN,denver,2018-01-17 12:00:00,2018
583,2017767521,1102,11/8/2017 8:00:00 PM,sex-aslt-rape,sexual-assault,NaN,NaN,denver,2017-11-08 20:00:00,2017


In [8]:
# With under 0.8266% NaNs, remove rows with NaN in Latitude/Longitude
data = data.dropna()
data

,INCIDENT_ID,OFFENSE_CODE,FIRST_OCCURRENCE_DATE,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,latitude,longitude,city,date,years
0,2016376978,5213,6/15/2016 11:31:00 PM,weapon-unlawful-discharge-of,all-other-crimes,39.773188,-104.809881,denver,2016-06-15 23:31:00,2016
1,20186000994,2399,10/11/2017 12:30:00 PM,theft-other,larceny,39.785649,-104.781434,denver,2017-10-11 12:30:00,2017
2,20166003953,2305,3/4/2016 8:00:00 PM,theft-items-from-vehicle,theft-from-motor-vehicle,39.663490,-104.957381,denver,2016-03-04 20:00:00,2016
3,201872333,2399,1/30/2018 7:20:00 PM,theft-other,larceny,39.702698,-104.941440,denver,2018-01-30 19:20:00,2018
4,2017411405,2303,6/22/2017 8:53:00 PM,theft-shoplift,larceny,39.717107,-104.955370,denver,2017-06-22 20:53:00,2017
5,201872837,5499,1/31/2018 12:44:00 AM,traf-other,all-other-crimes,39.743149,-104.961928,denver,2018-01-31 00:44:00,2018
6,20186001015,2304,6/1/2017 12:15:00 PM,theft-parts-from-vehicle,theft-from-motor-vehicle,39.732790,-105.025543,denver,2017-06-01 12:15:00,2017
7,201870628,5707,1/30/2018 7:40:00 AM,criminal-trespassing,all-other-crimes,39.723424,-104.983794,denver,2018-01-30 07:40:00,2018
8,201870833,5401,1/30/2018 9:10:00 AM,traffic-accident-hit-and-run,traffic-accident,39.736863,-104.991650,denver,2018-01-30 09:10:00,2018
9,20186001048,2305,1/31/2018 12:55:00 AM,theft-items-from-vehicle,theft-from-motor-vehicle,39.757627,-105.015451,denver,2018-01-31 00:55:00,2018


In [ ]:
# data_2014 = pd.DataFrame()
# data_2015 = pd.DataFrame()
# data_2016 = pd.DataFrame()
# data_2017 = pd.DataFrame()
# data_2018 = pd.DataFrame()

# data_2014 = data.loc[data["years"] == 2014]
# print(data_2014)
# data_2015 = data.loc[data["years"] == 2015]
# print(data_2015)
# data_2016 = data.loc[data["years"] == 2016]
# print(data_2016)
# data_2017 = data.loc[data["years"] == 2017]
# print(data_2017)
# data_2018 = data.loc[data["years"] == 2018]
# data_2018


In [9]:
# def reformat_datetime(df):
#     datetimes = []
#     for d,t  in zip(df["date"], df["Time Occurred"]):
# #         print(d)
# #         print(type(d))
#     #     datetime = (datetime.strftime(d, format='%Y-%m-%d') + " " + str(t))
#         d = datetime.strptime(str(d), "%Y-%m-%d %H:%M:%S")
#         date_time = datetime.strftime(d, format='%Y-%m-%d') + "T" + str((int(t/100))) + ":00:00"
#     #     datetime = d + " " + t
#         datetimes.append(date_time)
#     df["date_time"] = datetimes
#     return df

# def reformat_datetime(df):
#     datetimes = []
#     for d in df["date"]:
#         date_time = datetime.strftime(d, format='%Y-%m-%dT%H:00:00')
#         datetimes.append(date_time)
#     df["date_time"] = datetimes
#     return df

def reformat_datetime(df):
    dates = []
    times = []
    for d in df["date"]:
        dy = datetime.strftime(d, format='%Y-%m-%d')
        ti = datetime.strftime(d, format='%H')
        dates.append(dy)
        times.append(ti)
    df["date"] = dates
    df["time"] = times
    return df

In [10]:
data_denver = reformat_datetime(data)
data_denver

/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,INCIDENT_ID,OFFENSE_CODE,FIRST_OCCURRENCE_DATE,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,latitude,longitude,city,date,years,time
0,2016376978,5213,6/15/2016 11:31:00 PM,weapon-unlawful-discharge-of,all-other-crimes,39.773188,-104.809881,denver,2016-06-15,2016,23
1,20186000994,2399,10/11/2017 12:30:00 PM,theft-other,larceny,39.785649,-104.781434,denver,2017-10-11,2017,12
2,20166003953,2305,3/4/2016 8:00:00 PM,theft-items-from-vehicle,theft-from-motor-vehicle,39.663490,-104.957381,denver,2016-03-04,2016,20
3,201872333,2399,1/30/2018 7:20:00 PM,theft-other,larceny,39.702698,-104.941440,denver,2018-01-30,2018,19
4,2017411405,2303,6/22/2017 8:53:00 PM,theft-shoplift,larceny,39.717107,-104.955370,denver,2017-06-22,2017,20
5,201872837,5499,1/31/2018 12:44:00 AM,traf-other,all-other-crimes,39.743149,-104.961928,denver,2018-01-31,2018,00
6,20186001015,2304,6/1/2017 12:15:00 PM,theft-parts-from-vehicle,theft-from-motor-vehicle,39.732790,-105.025543,denver,2017-06-01,2017,12
7,201870628,5707,1/30/2018 7:40:00 AM,criminal-trespassing,all-other-crimes,39.723424,-104.983794,denver,2018-01-30,2018,07
8,201870833,5401,1/30/2018 9:10:00 AM,traffic-accident-hit-and-run,traffic-accident,39.736863,-104.991650,denver,2018-01-30,2018,09
9,20186001048,2305,1/31/2018 12:55:00 AM,theft-items-from-vehicle,theft-from-motor-vehicle,39.757627,-105.015451,denver,2018-01-31,2018,00





----

# Final view of date_time into weather data datetime format

In [11]:
data_denver.rename(columns={"OFFENSE_CODE": "code"}, inplace=True)
data_denver = data_denver[["city", "code", "date", "time", "latitude", "longitude"]].copy()
# data_denver.to_csv (r'data/Denver/data_denver.csv', index = None, header=True)

data_denver

/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/frame.py:4218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


,city,code,date,time,latitude,longitude
0,denver,5213,2016-06-15,23,39.773188,-104.809881
1,denver,2399,2017-10-11,12,39.785649,-104.781434
2,denver,2305,2016-03-04,20,39.663490,-104.957381
3,denver,2399,2018-01-30,19,39.702698,-104.941440
4,denver,2303,2017-06-22,20,39.717107,-104.955370
5,denver,5499,2018-01-31,00,39.743149,-104.961928
6,denver,2304,2017-06-01,12,39.732790,-105.025543
7,denver,5707,2018-01-30,07,39.723424,-104.983794
8,denver,5401,2018-01-30,09,39.736863,-104.991650
9,denver,2305,2018-01-31,00,39.757627,-105.015451


In [ ]:
# df_years = [data_2014, data_2015, data_2016, data_2017, data_2018]

# for i, yr in enumerate(df_years):
#     df = reformat_datetime(yr)
#     if i == 0:
#         data_2014_1 = df.copy()
#     elif i ==1:
#         data_2015_1 = df.copy()
#     elif i ==2:
#         data_2016_1 = df.copy()
#     elif i ==3:
#         data_2017_1 = df.copy()
#     elif i ==4:
#         data_2018_1 = df.copy()





----

# Final DFs per year

In [ ]:
data_2014_1

In [ ]:
data_2015_1

In [ ]:
data_2016_1

In [ ]:
data_2017_1

In [ ]:
data_2018_1

In [ ]:
# .to_csv(index=False)
data_2014_1.to_csv (r'data/Denver/data_2014.csv', index = None, header=True)
data_2015_1.to_csv (r'data/Denver/data_2015.csv', index = None, header=True)
data_2016_1.to_csv (r'data/Denver/data_2016.csv', index = None, header=True)
data_2017_1.to_csv (r'data/Denver/data_2017.csv', index = None, header=True)
data_2018_1.to_csv (r'data/Denver/data_2018.csv', index = None, header=True)